In [86]:
import os;

import sys;
sys.path.insert(0,os.getcwd() + '\\..\\lib');

import numpy as np
import scipy.io as spio;
from random import randint;
import matplotlib.pyplot as plot
from scipy.optimize import fmin_bfgs;

def displayImage(X):
    
    # fname = 'image.png'
    # image = Image.open(fname).convert("L")
    # arr = np.asarray(image)

    plot.imshow(X, cmap='gray')
    plot.show()

# displayImage(np.reshape(X[0],(20,20)));

class NeuralNetwork:
    
    def __init__(self, X, y, lam, yClasses, hiddenLayerSizes):
        
        self.X = X;
        self.lam = lam;
        self.m = X.shape[0];
        self.n = X.shape[1];
                                                  
        y1 = np.zeros((self.m, yClasses));
#         print(
        
#             y1.shape,
#             y,
#             y[0]%10
            
#         );
        
#         try:
        for i in range(0,self.m):
#             print(i, (y[i] % 10));
            y1[i, (y[i] % 10)] = 1;
        self.y = y1;
#         except:
#             print(i);
        
        
        
#         self.X = NeuralNetwork.addIntercept(self.X);
#         print(
#             [self.n],
#             hiddenLayerSizes,
#             [yClasses]
#         );
        thetasDimensions = [self.n] + hiddenLayerSizes + [yClasses];
#         print(
#             thetasDimensions
#         );
        self.Thetas = [];
        self.nn_params  = [];
        for i in range(0, len(thetasDimensions)-1):
            theta = self.initializeRandomWeights(thetasDimensions[i], thetasDimensions[i+1]);
            self.Thetas.append(theta);
            if len(self.nn_params) == 0:
                self.nn_params = theta.flatten();
            self.nn_params = np.hstack([self.nn_params, theta.flatten()]);
                
#         print(
#             self.Thetas
#         );

    @staticmethod
    def addIntercept(X):
        m = X.shape[0];
        X = np.hstack((np.ones((m,1)), X));
        return X;       
    
    @staticmethod
    def sigmoid(z):
        return 1 / (1 + np.exp(-1 * z));

    @staticmethod
    def sigmoidGradient(z):
        return np.multiply(LR.sigmoid(z), np.subtract( 1, LR.sigmoid(z)));
    
    @staticmethod
    def initializeRandomWeights(lIn, lOut):
        epsilonInit = 0.12;
        w = np.zeros((lOut, lIn + 1));
        w = np.random.rand(lOut, lIn + 1);
        w = np.subtract( np.multiply( w, (2.0 * epsilonInit) ), epsilonInit );
        return w;
    
    def costFunction(self, nn_params):

        m = self.X.shape[0];
        
        
        a = np.transpose(self.X);
        self.a = [];
        self.z = [];
        regularizationSum = 0;
        
#         a1 = np.transpose(LR.addIntercept(X));
#         z2 = np.dot(theta1, a1);
#         a2 = LR.sigmoid(z2);
#         a2 = np.transpose(LR.addIntercept(np.transpose(a2)));
#         z3 = np.dot(theta2, a2);
#         a3 = LR.sigmoid(z3);

        startIndex = 0;
        for i in range(0, len(self.Thetas)):
            endIndex = startIndex + (self.Thetas[i].shape[0] * self.Thetas[i].shape[1]);
            self.Thetas[i] = np.reshape(nn_params[startIndex:endIndex], self.Thetas[i].shape);
            startIndex = endIndex;
            
            print(self.Thetas[i].shape, np.transpose(NeuralNetwork.addIntercept(np.transpose(a))).shape);
            z = np.dot(self.Thetas[i], np.transpose(NeuralNetwork.addIntercept(np.transpose(a))));
            print("appended z", z.shape);
            self.z.append(z);
            a = NeuralNetwork.sigmoid(z);
            print("appended a", a.shape);
            self.a.append(a);
      
            regularizationSum += np.sum(
                    np.square(
                        np.hstack( [np.zeros((self.Thetas[i].shape[0],1)), self.Thetas[i][:, 1:]] )
                    )
            );
        
        a = np.transpose(a);
        J = np.add(
                np.multiply(
                    (1/self.m) 
                    , np.sum( 
                            np.subtract(
                                np.multiply(
                                    np.multiply(-1, self.y),
                                    np.log(a)
                                )
                                , np.multiply(
              
                                    np.subtract(1, self.y), 
                                    np.log(np.subtract(1, a))
                                )
                            )
                        )
                   )  
                , (
                    (self.lam/(2 * self.m))  
                    * regularizationSum
                )
        );
        return J;
                                                  
    def gradFunction(self, nn_params):
                                                  
#         Thetas = [i];
        
#         startIndex = 0;
#         for i in range(0, len(self.Thetas)):
#             endIndex = startIndex + (self.Thetas[i].shape[0] * self.Thetas[i].shape[1]);
#             Thetas[i] = np.reshape(nn_params[startIndex:endIndex], self.Thetas[i].shape);
#             startIndex = endIndex;
        
#         delta3 = np.subtract(a3, np.transpose(y1));
        self.Deltas = [];
        currDelta = np.subtract(self.a[-1], np.transpose(self.y));
        self.Deltas.append( currDelta );
        
#         delta2 = np.multiply( np.dot(np.transpose(theta2), delta3)[1:, :], sigmoidGradient(z2));
        for i in reversed(range(1, len(self.a)-1)):
            currDelta = np.multiply( np.dot(np.transpose(self.Thetas[i]), currDelta)[1:, :], sigmoidGradient(self.z[1]) );
            self.Deltas.append(currDelta);
        
        self.Deltas = list(reversed(self.Deltas));
#         theta1Grad = np.zeros(theta1Shape);
#         theta2Grad = np.zeros(theta2Shape);
#         theta1Grad = theta1Grad + np.dot(delta2, np.transpose(a1));
#         theta2Grad = theta2Grad + np.dot(delta3, np.transpose(a2));
#         theta1Grad = np.divide(theta1Grad, m) + np.hstack( [np.zeros((theta1.shape[0],1)), np.multiply((lam/m), theta1[:, 1:])] );
#         theta2Grad = np.divide(theta2Grad, m) + np.hstack( [np.zeros((theta2.shape[0],1)), np.multiply((lam/m), theta2[:, 1:])] );
        self.ThetasGrad = [];
        for i in range(0, len(self.Thetas)):
#             self.ThetasGrad.append(np.zeros(self.Thetas[i].shape));
            thetaGrad = np.dot(self.Deltas[i], np.transpose(self.a[i]));
            thetaGrad = np.divide(
                thetaGrad
                , self.m
            ) 
            + np.hstack(
                [np.zeros((thetaGrad.shape[0],1))
                 , np.multiply(
                     (self.lam/self.m)
                     , thetaGrad[:, 1:]
                 )
                ] 
            );
            self.ThetasGrad.append(thetaGrad);
        
        nn_params = [];
        for theta in self.ThetasGrad:
            if len(nn_params)==0:
                nn_params = theta.flatten();
                continue;
            nn_params = np.hstack([nn_params, theta.flatten()]);
        
        return nn_params;
         
    def buildModel(self):
        
        fminOutput = fmin_bfgs(
                            self.costFunction,
                            self.nn_params,
                            self.gradFunction,
                            disp=True,
                            maxiter=400,
                            full_output = True,
                            retall=True
                        );
        opTheta = fminOutput[0];
        return opTheta;
        
    


data1 = spio.loadmat("C:/Dinesh/Projects/Learning/machinelearning_coursera/ex4/ex4data1.mat");
X = data1["X"];
y = data1["y"];

# data1 = spio.loadmat("C:/Dinesh/Projects/Learning/machinelearning_coursera/ex4/ex4weights.mat");
# theta1 = data1["Theta1"];
# theta2 = data1["Theta2"];

# theta1 = initializeRandomWeights(n, hiddenLayerSize);
# theta2 = initializeRandomWeights(hiddenLayerSize, yClasses)

# nn_params = np.hstack([theta1.flatten(), theta2.flatten()])
# print(
#     theta1.shape,
#     theta2.shape,
#     nn_params.shape
    
# )

# m = X.shape[0];
# n = X.shape[1];
# hiddenLayerSize = 25;
# yClasses = 10;
# lam = 0;

# J = costFunction(nn_params, n, hiddenLayerSize, yClasses, X, y, lam);


# print(

#     X[1,:].shape, y[1].shape

# );
J = NeuralNetwork(X[1:4,:], y[1:4], 0.1, 10, [25]).buildModel();
# nn_params = [];
# for theta in J.Thetas:
#     if len(nn_params)==0:
#         nn_params = theta.flatten();
#         continue;
#     nn_params = np.hstack([nn_params, theta.flatten()]);
# cost = J.costFunction(nn_params);
# print("cost", cost);
print("cost", J);
# print(prev.shape)
                     

# fminOutput = fmin_bfgs(
#                             costFunction,
#                             nn_params,
#                             gradFunction,
#                             disp=True,
#                             maxiter=400,
#                             full_output = True,
#                             retall=True
#                         );

AttributeError: 'NeuralNetwork' object has no attribute 'a'

In [83]:
test = [1,2,3];
test = list(reversed(test));
test

[3, 2, 1]